In [ ]:
# Importamos las librerías necesarias

# Tratamiento de datos
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
import seaborn as sns
import matplotlib.pyplot as plt

# Librería para convertir palabras a numeros
from word2number import w2n

# Configuración para poder visualizar todas las columnas y filas de los DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

# Fase 2: Transformación de los datos.

In [ ]:
# Leo archivo CSV
hr_raw = pd.read_csv("HR RAW DATA.csv", index_col=0)

# Lo guardo en un DataFrame
df_hr_raw = pd.DataFrame(hr_raw)

# Hago una copia del DataFrame que iré limpiando
df_limpia = df_hr_raw.copy()

GENERAL
- ✔️Estandarizar strings
    - `BusinessTravel` (object) tiene guiones medios y bajos, estandarizar a guion bajo
    - `MaritalStatus` (object) revisar valores con faltas ortograficas
- ✔️Revisar que los que tienen que ser numericos no son string u otro tipo
    - `Age` (object) cambiar object a valores numéricos (por ej: thirty -> 30)
    - `DailyRate` (object) quitar simbolo dolar y pasar a valor numerico, comprobar si tienen decimales para decidir si INT o FLOAT. También tener en cuenta que hay strings que ponene 'nan'
    - `HourlyRate` (object) revisar que hacer con los string `Not available`
    - `MonthlyIncome`  (object) revisar que hacer con los string 'nan', ver si poner FLOAT o INT
    - `PerformanceRating`  (object) revisar que hacer con los string 'nan'
    - `TOTALWORKINGYEARS` (object) revisar que hacer con los string 'nan'
    - `WORKLIFEBALANCE` (object) revisar que hacer con los string 'nan'
    - `YearsInCurrentRole` (object) revisar que hacer con los string 'nan'
- ✔️Estandarizar valores numericos
    - `DistanceFromHome`(int64) tiene valores negativos, quitar
- ✔️Estandarizar booleanos o valores binarios
    - `Gender` (int64) cambiar 0 y 1 a algo mas legible
    - `RemoteWork` (object) estandarizar valores
- Revisar que hacer con los string 'nan', campos vacíos o nulos
    - Columnas anteriores
    - `employeenumber` (object) revisar que hacer con los string 'nan'

## 1. Primer filtro de eliminación de columnas

Elimino las columnas que no aportan información relevante o correcta antes de la primera limpieza, ya que la labor de limpieza puede ser bastante tediosa para ciertas columnas que no son necesarias. Las columnas a revisar antes de eliminar son:

- `Department` pocos datos, revisar si eliminar (MANTENER DE MOMENTO)
- `employeecount` todos con 1
- `Over18` solo hay Y y vacíos, revisar si es necesaria (ya hay una columna con la edad `Age`)
- `StandardHours` son 80 o nan, revisar si es necesaria
- `Salary` tiene un único valor para todos los registros, 1000000000$
- `RoleDepartament` eliminar, información redundantes (ya está el Role y el Department en otras columnas)
- `NUMBERCHILDREN` todas vacías, eliminar
- `YearsInCurrentRole` alto porcentaje de nulos
- `SameAsMonthlyIncome` igual que `MonthlyIncome`

- `employeecount`: todos los employees son únicos, no se necesita un conteo. La columna no aporta ninguna informacion relevante por lo que se podría eliminar.

In [ ]:
print(f"Los valores únicos de la columna 'employeecount' son: {df_limpia['employeecount'].unique()}")
print(f"Y el porcentaje de nulos es: {df_limpia['employeecount'].isnull().sum() / df_limpia.shape[0]}")

- `Over18`: La edad mínima en la columna `Age` es 18 y no tiene nulos, por lo que todos los employees son mayores de edad. La columna `Over18` debería de tener el valor `Y` (yes) para todas las filas, pero no aporta ninguna información relevante, por lo que también se podría eliminar.

In [ ]:
print(f"Los valores únicos de la columna 'Over18' son: {df_limpia['Over18'].unique()}")
print(f"Y el porcentaje de nulos de 'Over18' es: {round(df_limpia['Over18'].isnull().sum() / df_limpia.shape[0],2)}")
print(f"La edad mínima en la columna 'Age' es: {df_limpia['Age'].min()}")
print(f"Y el porcentaje de nulos de 'Age' es: {round(df_limpia['Age'].isnull().sum() / df_limpia.shape[0],2)}")

- `StandardHours`: El porcentaje de nulos en esta columna es muy alta y el único valor que tiene diferente al nulo es de 80. Esta columna no aporta información suficiente para poder imputarla, la vamos a eliminar.

In [ ]:
print(f"Los valores únicos de la columna 'StandardHours' son: {df_limpia['StandardHours'].unique()}")
print(f"Y el porcentaje de nulos de 'StandardHours' es: {round(df_limpia['StandardHours'].isnull().sum() / df_limpia.shape[0],2)}")

- `Salary`: Tiene un unico valor, 1000000000$. Es un valor demasiado elevado y es el mismo para todos los employees (no hay nulos), por lo que deducimos que son valores erróneos. Eliminaríamos esta columna también por incongruencia.

In [ ]:
print(f"Los valores únicos de la columna 'Salary' son: {df_limpia['Salary'].unique()}")
print(f"Y el porcentaje de nulos de 'Salary' es: {round(df_limpia['Salary'].isnull().sum() / df_limpia.shape[0],2)}")

- `RoleDepartament`: Esta columna repite la misma información que `Department` y `JobRole` y las combina (nos dan la misma información expresada de forma diferente), pero tiene un alto porcentaje de nulos, el mismo que `Department`. La información que aporta es redundante e incluso insuficiente por la cantidad de nulos, así que también la eliminaríamos.

In [ ]:
print(f"Y el porcentaje de nulos de 'RoleDepartament' es: {round(df_limpia['RoleDepartament'].isnull().sum() / df_limpia.shape[0],2)}")
print(f"Es el mismo porcentaje nulos que 'Department' es: {round(df_limpia['Department'].isnull().sum() / df_limpia.shape[0],2)}\n")
print(f"Un ejemplo de datos repetidos sería la fila 1428\n{df_limpia.loc[1428,['JobRole','Department','RoleDepartament']]}\n")
print(f"Un ejemplo de datos nulos sería la fila 0\n{df_limpia.loc[0,['JobRole','Department','RoleDepartament']]}")

- `NUMBERCHILDREN`: Todos los valores de esta columna son nulos, no aporta ninguna información por lo que la eliminaríamos.

In [ ]:
print(f"El porcentaje de nulos de 'NUMBERCHILDREN' es: {round(df_limpia['NUMBERCHILDREN'].isnull().sum() / df_limpia.shape[0],2)}")

- `YearsInCurrentRole` alto porcentaje de nulos, no aporta información suficiente para imputarla, la eliminaríamos.

In [ ]:
print(f"El porcentaje de nulos de 'YearsInCurrentRole' es: {round(df_limpia['YearsInCurrentRole'].isnull().sum() / df_limpia.shape[0],2)}")

- `SameAsMonthlyIncome`: Todos los valores de esta columna son iguales a los de `MonthlyIncome`, la información es redundante y se podría eliminar.

In [ ]:
# Comparo las columnas 'SameAsMonthlyIncome' y 'MonthlyIncome' para comprobar si son iguales
son_iguales = df_limpia['SameAsMonthlyIncome'].equals(df_limpia['MonthlyIncome'])
print(f"Las columnas 'SameAsMonthlyIncome' y 'MonthlyIncome' son iguales: {son_iguales}")

En este primer filtro elimino las columnas anteriormente analizadas, ya que he comprobado que son redundantes o no aportan información suficiente.

In [ ]:
columnas_a_eliminar = ['employeecount','Over18','StandardHours','Salary','RoleDepartament','NUMBERCHILDREN','YearsInCurrentRole','SameAsMonthlyIncome']

# Elimino las columnas de df_limpia, ya que es la copia del DataFrame que estoy limpiando
df_limpia.drop(columnas_a_eliminar, axis = 1, inplace = True)
df_limpia.head(3)

## 2. Homogeneizamos el nombre de las columnas

In [ ]:
nombres_snake_case = {
    'employeenumber': 'EmployeeNumber',
    'NUMCOMPANIESWORKED': 'NumCompaniesWorked',
    'TOTALWORKINGYEARS': 'TotalWorkingYears',
    'WORKLIFEBALANCE': 'WorkLifeBalance',
    'YEARSWITHCURRMANAGER': 'YearsWithCurrManager',
}

# Renombro las columnas para que todas estén en SnakeCase y sobreescribo el DataFrame usando el método rename
df_limpia.rename(columns=nombres_snake_case, inplace=True)
df_limpia.sample(3)

## 3. Homogeneizamos los valores de las columnas

### 3.1 Estandarización valores numéricos


#### 3.1.1 Columnas con valores categóricos que deberían de ser numéricas:

- `Age`                          object -> num

- `DailyRate`                    object -> num

- `HourlyRate`                   object -> num

- `MonthlyIncome`                object-> num

- `PerformanceRating`            object -> num

- `TOTALWORKINGYEARS`            object -> num

- `WORKLIFEBALANCE`              object -> num

- `YearsInCurrentRole`           object -> num

- `Age`: Esta columna debería de ser numérica pero contiene algunos valores que son strings (numeros escritos). Creo una función para convertir las palabras a números, para ello me he instalado la librería `from word2number import w2n`

In [ ]:
print(f"Los valores únicos de la columna 'Age' son: {df_limpia['Age'].unique()}\n")
print(f"Y los valores nulos que hay en la columna son: {df_limpia['Age'].isnull().sum()}")

In [ ]:
# Función para convertir palabras a números
def palabra_a_numero(age):
    
    # Intento convertir palabra a numero
    try:
        return w2n.word_to_num(age)
    # Si me da error, es que ya es un numero que se puede convertir de str a int
    except ValueError:
        return int(age)
        
# Aplicamos la función a la columna 'Age'
df_limpia['Age'] = df_limpia['Age'].apply(palabra_a_numero)

# Verificamos que ya no hay palabras
print(df_limpia['Age'].unique())

- `DailyRate`, `HourlyRate`, `MonthlyIncome`, `TotalWorkingYears`, `WorkLifeBalance`, `PerformanceRating` son columnas que incluyen valores numericos decimales pero que aparecen como tipo string en el DataFrame. Estos valores se pueden convertir en numero enteros. Habría que sustituir la coma por un punto para poder pasar la cifra a float y después a integer, así como gestionar los 'nan' que son string para convertirlos en NaN.

    Hay que tener en cuenta ciertas particularidades, ya que `DailyRate` tiene el símbolo dólar y los valores nulos de `HourlyRate` aparecen como 'Not available', 

In [ ]:
print(f"Los valores únicos de la columna 'DailyRate' son strings: {df_limpia['DailyRate'].unique()}")
print(f"Los valores únicos de la columna 'HourlyRate' son strings: {df_limpia['HourlyRate'].unique()}")
print(f"Los valores únicos de la columna 'MonthlyIncome' son strings: {df_limpia['MonthlyIncome'].unique()}")
print(f"Los valores únicos de la columna 'PerformanceRating' son strings: {df_limpia['PerformanceRating'].unique()}\n")
print(f"Los valores únicos de la columna 'TotalWorkingYears' son strings: {df_limpia['TotalWorkingYears'].unique()}\n")
print(f"Los valores únicos de la columna 'WorkLifeBalance' son strings: {df_limpia['WorkLifeBalance'].unique()}")

In [ ]:
def objeto_a_numero(cadena):
    if isinstance(cadena, str):
        cadena = cadena.replace('$', '').replace(',', '.')  # Reemplazar comas por puntos
        if cadena == 'nan' or cadena == 'Not Available':  # Manejar el caso de 'nan'
            return np.nan
        else:
            return float(cadena)  # Convertir a float y luego a int si no es 'nan'
    else:
        return cadena  # Devolver el valor original si no es una cadena
    
# Guardo en una lista las columnas que quiero convertir a enteros
col_a_enteros = ['DailyRate','HourlyRate','MonthlyIncome','TotalWorkingYears','WorkLifeBalance','PerformanceRating']

# Recorro la lista para aplicar a cada columna la funcion
for col in col_a_enteros:
    df_limpia[col] = df_limpia[col].apply(objeto_a_numero).astype('Int64')

In [ ]:
df_limpia[col_a_enteros].dtypes


In [ ]:
df_limpia[col_a_enteros].sample(5)

    En pandas, el tipo Int64 es un tipo de entero que soporta valores nulos (NaN), y cuando se usa este tipo, los valores nulos se representan como <NA> en lugar de NaN. 

#### 3.1.2 Valores numéricos inconsistentes

- La columna `DistanceFromHome` tiene valores negativos.

In [ ]:
print(f"Los valores únicos de la columna 'DistanceFromHome' son strings: {df_limpia['DistanceFromHome'].unique()}\n")
print(f"Los valores nulos de la columna 'DistanceFromHome' son strings: {df_limpia['DistanceFromHome'].isnull().sum()}")


In [ ]:
# Si la columna contiene valores numéricos, conviértelos a positivos si son negativos
df_limpia['DistanceFromHome'] = df_limpia['DistanceFromHome'].abs()

# Verifico si hay valores negativos en la columna 'DistanceFromHome'
hay_negativos = (df_limpia['DistanceFromHome'] < 0).any()

if hay_negativos:
    print("La columna 'DistanceFromHome' contiene valores negativos.")
else:
    print("La columna 'DistanceFromHome' no contiene valores negativos.")

In [ ]:
# Eliminar los valores negativos de la columna DistanceFromHome
#df_limpia['DistanceFromHome'] = df_limpia['DistanceFromHome'].apply(lambda x: x if x >= 0 else np.nan)

# Convertir DistanceFromHome a entero
#df_limpia['DistanceFromHome'] = df_limpia['DistanceFromHome'].astype('Int64')

### 3.2 Estandarizar booleanos o valores binarios

- `Gender` (int64) cambiar 0 y 1 a algo mas legible
- `RemoteWork` (object) estandarizar valores

- `Gender`: La columna `Gender` tiene valores de 0 y 1, los cuales son pocos intutitivos. Los reemplazo por "Male" y "Female", o "M" y "F" por ejemplo.

In [ ]:
print(f"Los valores únicos de la columna 'Gender' son: {df_limpia['Gender'].unique()}")

In [ ]:
# Reemplazo los valores de la columna Gender por Female y Male
df_limpia['Gender'] = df_limpia['Gender'].replace({0: 'Female', 1: 'Male'})
print(f"Después de la limpieza, los valores únicos de la columna 'Gender' son: {df_limpia['Gender'].unique()}")

- `RemoteWork`

In [ ]:
# Habría que estandarizar los datos 1/0, 'yes'/'no', 'false'/'true' del atributo RemoteWork
print(f"Los valores únicos de la columna 'RemoteWork' son: {df_limpia['RemoteWork'].unique()}")

In [ ]:
# Habría que estandarizar los datos 1/0, 'yes'/'no', 'false'/'true' del atributo RemoteWork
# Para una mayor legibilidad usamos los valores True/False en Remote Work
rw_datos_limpios = {
    'yes': True,
    '1': True,
    '0': False,
    'true': True,
    'false': False
}

df_limpia['RemoteWork'] = df_limpia['RemoteWork'].replace(rw_datos_limpios)
print(f"Después de la limpieza, los valores únicos de la columna 'RemoteWork' son: {df_limpia['RemoteWork'].unique()}")

### 3.3 Estandarización strings

- `BusinessTravel` (object) tiene guiones medios y bajos, estandarizar a guion bajo
- `Department`(object) tiene espacios al principio y al final, así como símbolo como '&'
- `JobRole`(object) iene espacios al principio y al final, y también letras en minusculas y en mayusculas
- `MaritalStatus` (object) revisar valores con faltas ortograficas

In [ ]:
print(f"Los valore únicos de la columna 'Attrition' son: {df_limpia['Attrition'].unique()}\n")
print(f"Los valore únicos de la columna 'BusinessTravel' son: {df_limpia['BusinessTravel'].unique()}\n")
print(f"Los valore únicos de la columna 'Department' son: {df_limpia['Department'].unique()}\n")
print(f"Los valore únicos de la columna 'EducationField' son: {df_limpia['EducationField'].unique()}\n")
print(f"Los valore únicos de la columna 'Gender' son: {df_limpia['Gender'].unique()}\n")
print(f"Los valore únicos de la columna 'JobRole' son: {df_limpia['JobRole'].unique()}\n")
print(f"Los valore únicos de la columna 'MaritalStatus' son: {df_limpia['MaritalStatus'].unique()}\n")

In [ ]:
df_limpia.tail()

In [ ]:
# Función para reemplazar guiones medios y los espacios por guiones bajos, poner todo en minuscula y sustituir el simbolo '&' por 'and'
def estandarizar_texto(value):
    if isinstance(value, str):
        return value.lower().strip().replace('-', '_').replace(' ','_').replace('&','and')
    return value

# Aplicar la función a todas las columnas
df_limpia= df_limpia.applymap(estandarizar_texto)

# Verificar el resultado
df_limpia.tail()

In [ ]:
print(f"Los valore únicos de la columna 'Attrition' son: {df_limpia['Attrition'].unique()}\n")
print(f"Los valore únicos de la columna 'BusinessTravel' son: {df_limpia['BusinessTravel'].unique()}\n")
print(f"Los valore únicos de la columna 'Department' son: {df_limpia['Department'].unique()}\n")
print(f"Los valore únicos de la columna 'EducationField' son: {df_limpia['EducationField'].unique()}\n")
print(f"Los valore únicos de la columna 'Gender' son: {df_limpia['Gender'].unique()}\n")
print(f"Los valore únicos de la columna 'JobRole' son: {df_limpia['JobRole'].unique()}\n")
print(f"Los valore únicos de la columna 'MaritalStatus' son: {df_limpia['MaritalStatus'].unique()}\n")

- En algunos valores de las columnas categóricas, por ejemplo, en la columna `MaritalStatus` en vez de "Married" en algunas filas aparece "Marreid". También aparecía 'Divorced' y 'divorced' como valores diferentes, pero con la función anterior ya se ha solucionado.

In [ ]:
print(f"Los valores únicos de la columna 'MaritalStatus' son: {df_limpia['MaritalStatus'].unique()}")

In [ ]:
# Uso el método replace y compruebo que se ha cambiado
df_limpia['MaritalStatus'] = df_limpia['MaritalStatus'].replace('marreid', 'married')
print(f"Después de la limpieza, los valores únicos de la columna 'MaritalStatus' son: {df_limpia['MaritalStatus'].unique()}")

## 4. Valores duplicados, insuficientes o incongruentes

### Eliminamos columnas con demasiados nulos

- Si el 80% de los registros son nulos eliminamos los atributos:

In [ ]:
# Calcula el porcentaje de valores nulos para cada atributo
porcentaje_nulos = df_limpia.isna().mean() * 100

# Filtra los atributos donde el porcentaje de nulos sea mayor al 80%
columnas_80_nulos = porcentaje_nulos[porcentaje_nulos > 80]

# Obtiene los nombres de los atributos 
columnas_a_eliminar = columnas_80_nulos.index.tolist()
print("Atributos con más del 80/%/ de valores nulos:")
print(columnas_a_eliminar)

In [ ]:
# Eliminamos las columnas seleccionadas
# df_limpia.drop(columns=columnas_a_eliminar, axis = 1, inplace=True)
# print(f"Las columnas {columnas_a_eliminar} han sido eliminadas")

In [ ]:
# Guardamos el DataFrame limpio en un nuevo archivo CSV
output_file_path = 'hr_data_limpio.csv'
df_limpia.to_csv(output_file_path, index=False)
print(f"El DataFrame modificado ha sido guardado en '{output_file_path}'.")